In [1]:
import os, sys, re, argparse, pdb, glob, pprint, json
import TS_pipe as tsp
import pandas as pd
f***REMOVED*** rdkit import Chem
f***REMOVED*** rdkit.Chem import rdmolfiles
f***REMOVED*** openbabel import pybel
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 200)
json_file = '../results/nbds_dHs.json'
df = pd.read_json(json_file, orient='index')
df = df.reset_index(names='Molecule')

In [2]:
#df = df[df['TS guess key value'] < 160.0]
#df = df.reset_index()
df

,Molecule,TS mode,key parameter atoms,TS guess key parameter,TS guess key value,pysis input conformers number,pysis output conformers,pysis output conformers number,pysis conformers properties,orca input conformers number,orca output conformers,orca output conformers number,orca conformers properties
0,C12H9NS,C1C=CCC=C1,"[3, 2, 1, 13, 10, 4]",distances C-C,"[2.257664586431065, 1.590449515957753]",2,"[C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': [2.259098553295914, 1.587785720644121]}, 'C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_co...",2,"[C12H9NS/9_C12H9NS_orca/cregened_conformers_0_reopt, C12H9NS/9_C12H9NS_orca/cregened_conformers_1_reopt]",2,"{'C12H9NS/9_C12H9NS_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -93.55801984, 'orca TS E corr': 0.13909014, 'orca TS def2-tzvp full energy': -915.879946131958, 'orca reac1 Gibbs en..."
1,C14H10N2O2,C1C=CCC=C1,"[3, 2, 1, 17, 14, 4]",distances C-C,"[2.243306434434007, 1.59091403159113]",1,[C14H10N2O2/8_C14H10N2O2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz],1,"{'C14H10N2O2/8_C14H10N2O2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz': {'TS conformer key value': [2.242666857508474, 1.589620528989925]}}",1,[C14H10N2O2/9_C14H10N2O2_orca/cregened_conformers_0_reopt],1,"{'C14H10N2O2/9_C14H10N2O2_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -137.29455632, 'orca TS E corr': 0.17366504, 'orca TS def2-tzvp full energy': -799.78347644756, 'orca reac1 Gi..."
2,C14H11N,C1C=CCC=C1,"[3, 4, 11, 14, 1, 2]",distances C-C,"[1.604106147791618, 2.221041776282837]",1,[C14H11N/8_C14H11N_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz],1,"{'C14H11N/8_C14H11N_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz': {'TS conformer key value': [1.6021877257919952, 2.220977958646366]}}",1,[C14H11N/9_C14H11N_orca/cregened_conformers_0_reopt],1,"{'C14H11N/9_C14H11N_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -95.94228704, 'orca TS E corr': 0.17407592, 'orca TS def2-tzvp full energy': -595.170502415932, 'orca reac1 Gibbs en..."
3,C16H11N,C1C=CCC=C1,"[3, 4, 13, 16, 1, 2]",distances C-C,"[2.25764632401464, 1.587709172998249]",1,[C16H11N/8_C16H11N_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz],1,"{'C16H11N/8_C16H11N_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz': {'TS conformer key value': [2.256761962947801, 1.5854435528399402]}}",1,[C16H11N/9_C16H11N_orca/cregened_conformers_0_reopt],1,"{'C16H11N/9_C16H11N_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -107.22802299, 'orca TS E corr': 0.17935694, 'orca TS def2-tzvp full energy': -671.331952380918, 'orca reac1 Gibbs e..."
4,C16H16N2,C1C=CCC=C1,"[3, 2, 1, 17, 14, 4]",distances C-C,"[1.6134685534967521, 2.209688715198768]",1,[C16H16N2/8_C16H16N2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz],1,"{'C16H16N2/8_C16H16N2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz': {'TS conformer key value': [1.611799443653453, 2.206974912419342]}}",1,[C16H16N2/9_C16H16N2_orca/cregened_conformers_0_reopt],1,"{'C16H16N2/9_C16H16N2_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -120.24879565, 'orca TS E corr': 0.24413369000000001, 'orca TS def2-tzvp full energy': -729.262260069406, 'orca re..."
5,C18H13N,C1C=CCC=C1,"[3, 2, 1, 18, 15, 4]",distances C-C,"[1.565046193888104, 2.244111055627198]",3,"[C18H13N/8_C18H13N_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C18H13N/8_C18H13N_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz, C18H13N/8_C18H13N_filtered_TS_irc/cre...",3,"{'C18H13N/8_C18H13N_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': [1.5623733581139851, 2.240605138105123]}, 'C18H13N/8_C18H13N_filtered_TS_irc/cregened_c...",3,"[C18

In [3]:
failed_orca_count = 0
failed_orca_list = []
conformers_by_orca_count = df['pysis output conformers number'].sum()
conformers_to_orca_count = df['orca input conformers number'].sum()
energies_dict = {}
for i in df.index:
    closed_Es = []
    TS_Es = []
    closed_Hs = []
    open_Hs = []
    for j, conf in enumerate(df.loc[i]['orca output conformers']):
        try:
            if not list(df.loc[i]['orca conformers properties'][conf].values()):
                failed_orca_count += 1
                failed_orca_list.append(conf)
            if all(df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'][j] < 1.8 for j in [0,1]):
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac1 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac1 E corr']
                closed_H = df.loc[i]['orca conformers properties'][conf]['orca reac1 enthalpy']
                open_H = df.loc[i]['orca conformers properties'][conf]['orca reac2 enthalpy']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'], 'selected as metastable')
            else:
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac2 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac2 E corr']
                closed_H = df.loc[i]['orca conformers properties'][conf]['orca reac2 enthalpy']
                open_H = df.loc[i]['orca conformers properties'][conf]['orca reac1 enthalpy']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac2 conformer key value'], 'selected as metastable')
            TS_energy =  df.loc[i]['orca conformers properties'][conf]['orca TS def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca TS E corr']
        except KeyError as e:
            print(f'Conformer {conf} is missing')
            print(e)
            continue
        closed_Es.append(closed_energy)
        TS_Es.append(TS_energy)
        closed_Hs.append(closed_H)
        open_Hs.append(open_H)
    if not ( closed_Es and TS_Es ):
        continue
    energies_dict[f"{df.loc[i]['Molecule']}"] = {}
    energies_dict[f"{df.loc[i]['Molecule']}"]['Closed energies'] = closed_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['TS energies'] = TS_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['open enthalpy'] = open_Hs
    energies_dict[f"{df.loc[i]['Molecule']}"]['closed enthalpy'] = closed_Hs
print(f'Conformers generated by ORCA stage: {conformers_to_orca_count}')
print(f'Failed ORCA jobs: {failed_orca_count}')
print(f'Failed ORCA list:', failed_orca_list)
pprint.pprint(energies_dict)
print(len(energies_dict.keys()))

with open('nbs_final_energies.json', 'w') as json_file:
    json.dump(energies_dict, json_file, indent=4)



Structure with  [1.507875024203176, 1.5080810086253211] selected as metastable
Structure with  [1.49932465814777, 1.505827945862857] selected as metastable
Structure with  [1.5021888056226271, 1.5071959229224632] selected as metastable
Structure with  [1.499783642275507, 1.506759857512999] selected as metastable
Structure with  [1.508377159671139, 1.505941133516008] selected as metastable
Structure with  [1.498269374928687, 1.506428680091123] selected as metastable
Structure with  [1.503505459324578, 1.507875833408987] selected as metastable
Structure with  [1.497188467119165, 1.505660983087073] selected as metastable
Structure with  [1.510328209518712, 1.5049116832971219] selected as metastable
Structure with  [1.499488972619511, 1.5038448385325052] selected as metastable
Structure with  [1.502537106993978, 1.5007242635983662] selected as metastable
Conformers generated by ORCA stage: 11
Failed ORCA jobs: 0
Failed ORCA list: []
{'C12H9NS': {'Closed energies': [-915.832534455697, -915.

In [4]:
os.getcwd()

'C:\\Users\\***REMOVED***pe672\\Documents\\GitHub\\TS_pipeline\\src'